In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import bz2

In [5]:
#Filenames
fsearches='searches.csv.bz2'
fbookings='bookings.csv.bz2'

In [ ]:
# Read the file line by line and count line by line
count=0
with bz2.BZ2File(fsearches,"r") as file:
    for line in file:
        count +=1
print count
nlines_searches = count
# Read the file line by line and count
count=0
with bz2.BZ2File(fbookings,"r") as file:
    for line in file:
        count +=1
print count
nlines_bookings=count

In [7]:
print "File %s has %s lines."%(fsearches,nlines_searches)
print "File %s has %s lines."%(fbookings,nlines_bookings)

File searches.csv.bz2 has 20390199 lines.
File bookings.csv.bz2 has 10000011 lines.
